In [1]:
import numpy as np

from datasets import BasicDataset
from models import XGBModel

In [36]:
ds = BasicDataset(
    ohe_features=[

    ],
    le_features=[

    ],
    categorical_features=[
        "CONTRACT_CREDIT_INTERMEDIARY",
        "CONTRACT_CURRENCY",
        "CONTRACT_FREQUENCY_TYPE",
        "CONTRACT_LOAN_CONTRACT_TYPE",
        "CONTRACT_LOAN_TYPE",
        "CONTRACT_MORTGAGE_TYPE",
        "CONTRACT_REFINANCED",
        "CONTRACT_TYPE_OF_INTEREST_REPAYMENT",
        "BORROWER_TYPE_OF_CUSTOMER",
        "BORROWER_TYPE_OF_SETTLEMENT",
        "CONTRACT_BANK_ID",
        "BORROWER_CITIZENSHIP",
        "BORROWER_COUNTRY",
        "BORROWER_COUNTY"
    ],
    log_transform_features=[

    ],
    numerical_features=[
        "CONTRACT_CREDIT_LOSS",
        "CONTRACT_LOAN_AMOUNT",
        "CONTRACT_RISK_WEIGHTED_ASSETS",
        "CONTRACT_INSTALMENT_AMOUNT_2",
        "CONTRACT_INTEREST_PERIOD",
        "CONTRACT_INTEREST_RATE",
        "CONTRACT_LOAN_TO_VALUE_RATIO",
        "BORROWER_BIRTH_YEAR",
    ],
    feature_engineered_cols=[]
    )

/Users/peter.nagy/repos/datarace23/datasets/base.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = df[feature].astype("category")
/Users/peter.nagy/repos/datarace23/datasets/base.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[feature] = df[feature].astype("category")
/Users/peter.nagy/repos/datarace23/datasets/base.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [37]:
ds.save()

'2023-11-22-16-49-b4bc1'

In [ ]:
# ohe + le: 2023-11-22-11-50-e739e
# categorical: 2023-11-22-15-48-2603b
# only le + num: 2023-11-22-16-41-3086b
# categorical + moar params: 2023-11-22-16-49-b4bc1

In [3]:
ds = BasicDataset.load(save_id='2023-11-22-16-43-ee0f3')

In [7]:
ds.train_df["CONTRACT_COUNT"] = ds.train_df.sort_values(["BORROWER_ID", "CONTRACT_DATE_OF_LOAN_AGREEMENT"]).BORROWER_ID.groupby(
            ds.train_df["BORROWER_ID"]
        ).cumcount()

In [9]:
ds.train_df.loc[ds.train_df["BORROWER_ID"]  == 'xNullx', "CONTRACT_COUNT"] = 0

In [10]:
ds.train_df.CONTRACT_COUNT.value_counts()

CONTRACT_COUNT
0     458316
1      57066
2       8654
3       1752
4        404
5        104
6         37
7         19
8         13
9          6
10         5
11         3
12         2
13         1
Name: count, dtype: int64

In [38]:
model = XGBModel(
{
        "n_estimators": 100,
        "max_depth": 10,
        "learning_rate": 0.1,
        "subsample": 0.5,
        "colsample_bytree": 0.5,
        "reg_alpha": 1,
        "reg_lambda": 1,
        "random_state": 42,
        "enable_categorical": True,
    }
)



In [39]:
model.fit(ds)

Cross-validation scores mean: 0.9939967453289092, std: 0.0002683622615886357
Test score: 0.9945097741218488
F1 score: 0.8670041417395307


0.9939967453289092

In [40]:
model.predict(ds).to_csv("submissions/categorical_moar_features.csv", index=False)

In [41]:
model.save()

'2023-11-22-16-50-68f8f'

In [4]:
import optuna

def objective(trial):
    model = XGBModel(
        {
            "n_estimators": trial.suggest_int("n_estimators", 50, 500),
            "max_depth": trial.suggest_int("max_depth", 5, 20),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "reg_alpha": trial.suggest_float("reg_alpha", 0.1, 10),
            "reg_lambda": trial.suggest_float("reg_lambda", 0.1, 10),
            "random_state": 42
        }
    )
    return model.fit(ds)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print(study.best_params)
print(study.best_value)


/Users/peter.nagy/repos/datarace23/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2023-11-22 12:20:19,445] A new study created in memory with name: no-name-95c86487-9d85-48ed-ab43-cb0eede5c3ce


Cross-validation scores mean: 0.9938848700397841, std: 0.00032630200312134557
Test score: 0.9943577955508274


[I 2023-11-22 12:20:35,313] Trial 0 finished with value: 0.863196683555965 and parameters: {'n_estimators': 81, 'max_depth': 8, 'learning_rate': 0.10394912277108934, 'subsample': 0.7749594535944224, 'colsample_bytree': 0.8482946891447991, 'reg_alpha': 6.119787361354072, 'reg_lambda': 0.7583983327348423, 'random_state': 88}. Best is trial 0 with value: 0.863196683555965.


F1 score: 0.863196683555965
Cross-validation scores mean: 0.9942394927339497, std: 0.0003037266413051975
Test score: 0.9948327285852695


[I 2023-11-22 12:20:55,042] Trial 1 finished with value: 0.8746543778801844 and parameters: {'n_estimators': 102, 'max_depth': 15, 'learning_rate': 0.09097065978116066, 'subsample': 0.9515278066804946, 'colsample_bytree': 0.9386878623783537, 'reg_alpha': 4.722760624017423, 'reg_lambda': 5.096329513857128, 'random_state': 30}. Best is trial 1 with value: 0.8746543778801844.


...

F1 score: 0.8833100883310088
Cross-validation scores mean: 0.9945497875829904, std: 0.0002045014093325593
Test score: 0.9950037044776686


[I 2023-11-22 13:34:38,268] Trial 98 finished with value: 0.8786340562990309 and parameters: {'n_estimators': 295, 'max_depth': 12, 'learning_rate': 0.15802389587327134, 'subsample': 0.9630004130471337, 'colsample_bytree': 0.5114355916831214, 'reg_alpha': 1.2833977124748874, 'reg_lambda': 9.597575639410788, 'random_state': 31}. Best is trial 16 with value: 0.8901303538175046.


F1 score: 0.8786340562990309
Cross-validation scores mean: 0.9947207661977681, std: 0.00028721751486960294
Test score: 0.9951936776914455


[I 2023-11-22 13:35:21,179] Trial 99 finished with value: 0.8831408775981523 and parameters: {'n_estimators': 242, 'max_depth': 13, 'learning_rate': 0.1659807042848357, 'subsample': 0.9843404442783942, 'colsample_bytree': 0.5510809395995141, 'reg_alpha': 0.5929469064517836, 'reg_lambda': 8.524471182981543, 'random_state': 93}. Best is trial 16 with value: 0.8901303538175046.


F1 score: 0.8831408775981523
{'n_estimators': 269, 'max_depth': 18, 'learning_rate': 0.1625455025741216, 'subsample': 0.8764174359821452, 'colsample_bytree': 0.606095076121955, 'reg_alpha': 0.5612075815264514, 'reg_lambda': 8.971726361946647, 'random_state': 57}
0.8901303538175046


In [7]:
# get best model from study
model = XGBModel(study.best_params)

In [8]:
model.fit(ds)

Cross-validation scores mean: 0.9946869930661473, std: 0.00029706197884966804
Test score: 0.9955166321548662
F1 score: 0.8901303538175046


0.8901303538175046

In [5]:
model.f1_score

0.8721251149954002

In [9]:
model.save()

'2023-11-22-13-54-e3f61'

In [4]:
model = model.load(save_id='2023-11-22-11-51-94461')

In [10]:
model.predict(ds).to_csv('submissions/submission_optuna1.csv', index=False)